In [6]:
import sys
import random
import numpy as np
import torch
import learn2learn as l2l
import pandas as pd

from torch import nn, optim
import torchvision
from torchvision import datasets, transforms
from PIL.Image import LANCZOS
from config import *

import json

In [2]:
torch.cuda.is_available()

True

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')

In [4]:
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.8.1013163132/pythonFiles/vscode_datascience_helpers',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.8.1013163132/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.8.1013163132/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python38.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/datasets-1.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [10]:
with open('../data/cub_meta-info.json') as file:
    cub_data = json.load(file)
    file.close()

In [5]:
from src.utils import Profiler

In [6]:
profiler = Profiler('MAML_{}_{}-way_{}-shot_{}-queries'.format('wtv', 5, 12091092, 223))

In [16]:
from data.loaders import Omniglotmix, MiniImageNet, CUBirds200, CIFARFS, FGVCAircraft, FGVCFungi, TieredImagenet, VGGFlower102
from data.taskers import gen_tasks
from src.zoo.archs import EncoderNN, MatchingNetwork
from src.zoo.matching_nets_utils import setup, logits, accuracy, inner_adapt_matching
#from src.utils import Profiler

In [18]:
TieredImagenet(root='/home/anuj/Desktop/Work/TU_Delft/research/implement/tiered_imagenet', mode='train', download=True)

KeyboardInterrupt: 

In [17]:
VGGFlower102(root='/home/anuj/Desktop/Work/TU_Delft/research/implement/vgg_flower', mode='all', download=True)

In [59]:
classes = list(range(1623))
random.shuffle(classes)
image_transforms = transforms.Compose([transforms.Resize(28, interpolation=LANCZOS),
                                                    transforms.ToTensor(),
                                                    lambda x: 1.0 - x,
                                                ])

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [60]:
train_tasks = gen_tasks('omniglot', '/home/anuj/Desktop/Work/TU_Delft/research/implement/omniglot', image_transforms=image_transforms, n_ways=5, k_shots=1, q_shots=15, classes=classes[:1100])

In [61]:
train_tasks.sample()[0].shape

torch.Size([80, 1, 28, 28])

In [18]:
task = train_tasks.sample()

In [6]:
train_tasks = gen_tasks(dataname='miniimagenet', root='../../mini_imagenet', mode='train', n_ways=5, k_shots=1, q_shots=15, image_transforms=None)

In [62]:
data1, labels1 = train_tasks.sample()

In [63]:
data1.shape

torch.Size([80, 1, 28, 28])

In [6]:
train_tasks, valid_tasks, test_tasks, learner = setup(
    'omniglot', '/home/anuj/Desktop/Work/TU_Delft/research/implement/omniglot', 5, 1, 15, 5, 1, 15, 1, 2, 'cpu')

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [21]:
ttask = train_tasks.sample()

In [38]:
def inner_adapt_matching_here(task, loss, learner, n_ways, k_shots, q_shots, EPSILON, device):
    
    

    preds = logits(queries=queries, support=support, EPSILON=EPSILON)
    attention = (-preds).softmax(dim=1)

    y_onehot = torch.zeros(n_ways * k_shots, n_ways).to(device)

    y = support_labels.unsqueeze(-1)
    y_onehot = y_onehot.scatter(1, y, 1)

    y_pred = torch.mm(attention, y_onehot.to(device))

    # Calculated loss with negative log likelihood
    # Clip predictions for numerical stability
    clipped_y_pred = y_pred.clamp(EPSILON, 1 - EPSILON)
    eval_loss = loss(clipped_y_pred.log(), queries_labels)
    eval_acc = accuracy(clipped_y_pred, queries_labels)

    return eval_loss, eval_acc


In [22]:
data, labels = ttask
data, labels = data.to(device), labels.to(device)
total = n_ways * (k_shots + q_shots)
queries_index = np.zeros(total)

data = learner.encoder(data)
# Extracting the evaluation datums from the entire task set, for the meta gradient calculation
for offset in range(n_ways):
    queries_index[np.random.choice(
        k_shots+q_shots, q_shots, replace=False) + ((k_shots + q_shots)*offset)] = True
support = data[np.where(queries_index == 0)]
support_labels = labels[np.where(queries_index == 0)]
queries = data[np.where(queries_index == 1)]
queries_labels = labels[np.where(queries_index == 1)]


In [23]:
support, _, _ = learner.support_encoder(support.unsqueeze(1))
support = support.squeeze(1)
queries = learner.query_encoder(queries, support, device)

In [76]:
preds = logits(queries=queries, support=support, EPSILON=1e-8)
attention = (-preds).softmax(dim=1)

In [25]:
queries.shape

torch.Size([5, 64])

In [39]:
inner_adapt_matching_here(ttask, nn.NLLLoss(), learner, 5, 1, 15, 1e-8, 'cpu')

ValueError: Expected input batch_size (5) to match target batch_size (75).

In [36]:
loss, acc = inner_adapt_matching(ttask, nn.NLLLoss(), learner, 5, 1, 15, 1e-8, 'cpu')

ValueError: Expected input batch_size (5) to match target batch_size (75).

In [78]:
model = MatchingNetwork(1, (2,2), True,
                 1, 64, 2, 'cpu')

In [8]:
data1, labels1 = train_tasks.sample()

In [9]:
a = learner.encoder(data1)

In [10]:
a.shape

torch.Size([80, 64])

In [11]:
labels1.shape

torch.Size([80])

In [12]:
device = 'cpu'
n_ways = 5; k_shots = 1; q_shots = 15

In [13]:
data, labels = a, labels1
data, labels = data.to(device), labels.to(device)
total = n_ways * (k_shots + q_shots)
queries_index = np.zeros(total)

# Extracting the evaluation datums from the entire task set, for the meta gradient calculation
for offset in range(n_ways):
    queries_index[np.random.choice(
        k_shots+q_shots, q_shots, replace=False) + ((k_shots + q_shots)*offset)] = True
support = data[np.where(queries_index == 0)]
support_labels = labels[np.where(queries_index == 0)]
queries = data[np.where(queries_index == 1)]
queries_labels = labels[np.where(queries_index == 1)]


In [14]:
support.shape

torch.Size([5, 64])

In [48]:
g = BidrectionalLSTM(1600, 1).to('cuda')

NameError: name 'BidrectionalLSTM' is not defined

In [15]:
support, _, _ = learner.support_encoder(support.unsqueeze(1))

In [16]:
support = support.squeeze(1)

In [17]:
support

tensor([[ 0.7751,  0.0515,  0.1438,  0.1257,  0.6151,  0.3853, -0.1882,  0.2459,
          0.2943,  0.1744,  1.0263,  0.2978, -0.0356,  2.6314,  1.1487,  1.4413,
         -0.3744,  0.4918,  0.3630,  1.3094, -0.5490,  0.1826,  1.0999,  2.8820,
          0.2817, -0.2709,  1.4906,  0.2703,  0.0196, -0.3053,  0.7905,  1.4817,
          0.3449,  1.4603,  2.4432,  0.1692,  0.5180,  0.0957,  0.4308, -0.1557,
          0.0240,  0.5268,  0.8025,  2.8366,  0.5357,  0.4689,  0.8396,  0.1500,
         -0.3126,  1.2549,  0.1347,  1.3268,  0.1980,  0.7349,  0.3823, -0.1134,
          0.8832,  0.5702,  0.1782, -0.0140,  0.9304,  0.8235,  0.0473,  0.1978],
        [ 0.5596,  0.0285,  0.4590,  0.4996,  0.5670,  0.3072, -0.3717,  0.5611,
          0.3285,  0.1010,  1.1196,  0.5308,  0.1131,  1.1376,  0.4775,  0.6012,
          0.0713,  0.5458, -0.0818,  1.1150, -0.3621,  0.0060,  0.6649,  1.3462,
          1.1295, -0.2163,  0.2017,  0.5394,  0.1455,  0.0228,  0.9664, -0.0309,
          0.5418,  0.7056, 

In [52]:
f = AttentionLSTM(1600, 2).to('cuda')

NameError: name 'AttentionLSTM' is not defined

In [18]:
queries = learner.query_encoder(support, queries, 'cpu')

In [20]:
queries.shape

torch.Size([75, 64])

In [19]:
preds = logits(queries=queries, support=support, EPSILON=1e-8)
preds.shape

torch.Size([75, 5])

In [55]:
attention = (-preds).softmax(dim=1)
attention.shape

torch.Size([75, 5])

In [56]:
y_onehot = torch.zeros(n_ways * k_shots, n_ways).to(device)
y_onehot.shape

torch.Size([5, 5])

In [57]:
y = support_labels.unsqueeze(-1)
y_onehot = y_onehot.scatter(1, y, 1)

In [58]:
support_labels

tensor([2, 3, 1, 4, 0])

In [59]:
y_onehot

tensor([[0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.]])

In [60]:
y_pred = torch.mm(attention, y_onehot.to(device))
y_pred

tensor([[0.2058, 0.2036, 0.1957, 0.2050, 0.1899],
        [0.1956, 0.1976, 0.2166, 0.2037, 0.1865],
        [0.2083, 0.1987, 0.2101, 0.1919, 0.1910],
        [0.2055, 0.2053, 0.2081, 0.1932, 0.1879],
        [0.2011, 0.1960, 0.2133, 0.2040, 0.1856],
        [0.2077, 0.2002, 0.2032, 0.2023, 0.1865],
        [0.1995, 0.2046, 0.2021, 0.2026, 0.1913],
        [0.1996, 0.2077, 0.2058, 0.2057, 0.1813],
        [0.1987, 0.1895, 0.2126, 0.2015, 0.1978],
        [0.1997, 0.2014, 0.2123, 0.2060, 0.1806],
        [0.2008, 0.2039, 0.2126, 0.2014, 0.1813],
        [0.2009, 0.2024, 0.2079, 0.2039, 0.1848],
        [0.2066, 0.2012, 0.2052, 0.1955, 0.1914],
        [0.2069, 0.2022, 0.2057, 0.1962, 0.1890],
        [0.2067, 0.2056, 0.1974, 0.2034, 0.1869],
        [0.1924, 0.1941, 0.2099, 0.2090, 0.1944],
        [0.1998, 0.2021, 0.1974, 0.2039, 0.1968],
        [0.2090, 0.1884, 0.2082, 0.1982, 0.1963],
        [0.1945, 0.1949, 0.2063, 0.2106, 0.1936],
        [0.2008, 0.2026, 0.1980, 0.2023, 0.1963],


In [61]:
queries_labels.shape

torch.Size([75])

In [62]:
clipped_y_pred = y_pred.clamp(1e-8, 1 - 1e-8)
clipped_y_pred

tensor([[0.2058, 0.2036, 0.1957, 0.2050, 0.1899],
        [0.1956, 0.1976, 0.2166, 0.2037, 0.1865],
        [0.2083, 0.1987, 0.2101, 0.1919, 0.1910],
        [0.2055, 0.2053, 0.2081, 0.1932, 0.1879],
        [0.2011, 0.1960, 0.2133, 0.2040, 0.1856],
        [0.2077, 0.2002, 0.2032, 0.2023, 0.1865],
        [0.1995, 0.2046, 0.2021, 0.2026, 0.1913],
        [0.1996, 0.2077, 0.2058, 0.2057, 0.1813],
        [0.1987, 0.1895, 0.2126, 0.2015, 0.1978],
        [0.1997, 0.2014, 0.2123, 0.2060, 0.1806],
        [0.2008, 0.2039, 0.2126, 0.2014, 0.1813],
        [0.2009, 0.2024, 0.2079, 0.2039, 0.1848],
        [0.2066, 0.2012, 0.2052, 0.1955, 0.1914],
        [0.2069, 0.2022, 0.2057, 0.1962, 0.1890],
        [0.2067, 0.2056, 0.1974, 0.2034, 0.1869],
        [0.1924, 0.1941, 0.2099, 0.2090, 0.1944],
        [0.1998, 0.2021, 0.1974, 0.2039, 0.1968],
        [0.2090, 0.1884, 0.2082, 0.1982, 0.1963],
        [0.1945, 0.1949, 0.2063, 0.2106, 0.1936],
        [0.2008, 0.2026, 0.1980, 0.2023, 0.1963],


In [69]:
loss = nn.NLLLoss()
eval_loss = loss(clipped_y_pred.log(), queries_labels)


In [70]:
eval_loss

tensor(1.5884, grad_fn=<NllLossBackward>)

In [30]:
eval_acc = accuracy(clipped_y_pred, queries_labels)

In [ ]:


# Unsqueeze to force y to be of shape (K*n, 1) as this is needed for .scatter()


# Calculated loss with negative log likelihood
# Clip predictions for numerical stability
eval_loss = loss(clipped_y_pred.log(), queries_labels)
eval_acc = accuracy(clipped_y_pred, queries_labels)

In [43]:
def logits(support, queries, EPSILON):
    # Module with cosine similarity

    n_queries = queries.shape[0]
    n_support = support.shape[0]

    normalised_queries = queries / (queries.pow(2).sum(dim=1, keepdim=True).sqrt() + EPSILON)
    normalised_support = support / (support.pow(2).sum(dim=1, keepdim=True).sqrt() + EPSILON)

    expanded_x = normalised_queries.unsqueeze(1).expand(n_queries, n_support, -1)
    expanded_y = normalised_support.unsqueeze(0).expand(n_queries, n_support, -1)

    logits = (expanded_x * expanded_y).sum(dim=2)
    return 1 - logits

In [52]:
attention = (-logits(support, queries, 0.00001)).softmax(dim=1)

In [53]:
y_onehot = torch.zeros(n_ways * k_shots, n_ways).to(device)

# Unsqueeze to force y to be of shape (K*n, 1) as this
# is needed for .scatter()
y = support_labels.unsqueeze(-1)
y_onehot = y_onehot.scatter(1, y, 1)

y_pred = torch.mm(attention, y_onehot.cuda().double())

# Calculated loss with negative log likelihood
# Clip predictions for numerical stability
clipped_y_pred = y_pred.clamp(0.0001, 1 - 0.0001)
#eval_loss = loss(clipped_y_pred.log(), queries_labels)


In [56]:
clipped_y_pred.argmax(dim=1)

tensor([0, 3, 0, 0, 3, 3, 1, 1, 1, 3, 0, 3, 1, 1, 2], device='cuda:0')

In [ ]:

    # Optionally apply full context embeddings
    if fce:
        # LSTM requires input of shape (seq_len, batch, input_size). `support` is of
        # shape (k_way * n_shot, embedding_dim) and we want the LSTM to treat the
        # support set as a sequence so add a single dimension to transform support set
        # to the shape (k_way * n_shot, 1, embedding_dim) and then remove the batch dimension
        # afterwards

        # Calculate the fully conditional embedding, g, for support set samples as described
        # in appendix A.2 of the paper. g takes the form of a bidirectional LSTM with a
        # skip connection from inputs to outputs
        support, _, _ = model.g(support.unsqueeze(1))
        support = support.squeeze(1)

        # Calculate the fully conditional embedding, f, for the query set samples as described
        # in appendix A.1 of the paper.
        queries = model.f(support, queries)

    # Efficiently calculate distance between all queries and all prototypes
    # Output should have shape (q_queries * k_way, k_way) = (num_queries, k_way)
    distances = pairwise_distances(queries, support, distance)

    # Calculate "attention" as softmax over support-query distances
    attention = (-distances).softmax(dim=1)

    # Calculate predictions as in equation (1) from Matching Networks
    # y_hat = \sum_{i=1}^{k} a(x_hat, x_i) y_i
    y_pred = matching_net_predictions(attention, n_shot, k_way, q_queries)

    # Calculated loss with negative log likelihood
    # Clip predictions for numerical stability
    clipped_y_pred = y_pred.clamp(EPSILON, 1 - EPSILON)
    loss = loss_fn(clipped_y_pred.log(), y)

    if train:
        # Backpropagate gradients
        loss.backward()
        # I found training to be quite unstable so I clip the norm
        # of the gradient to be at most 1
        clip_grad_norm_(model.parameters(), 1)
        # Take gradient step
        optimiser.step()

    return loss, y_pred


In [29]:
def logits(support, queries, n, k, q):
    prototypes = support.view(n, k, -1).mean(dim=1)
    a = queries.shape[0]
    b = prototypes.shape[0]
    logits = -((queries.unsqueeze(1).expand(a,b,-1) - prototypes.unsqueeze(0).expand(a,b,-1))**2).sum(dim=2)
    return logits

In [ ]:
support.view()

In [30]:
logits(support, queries, 30, 1, 15).shape 

torch.Size([450, 30])

In [67]:
queries_labels

tensor([1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 4, 4, 4, 4,
        4], device='cuda:0')

In [25]:
support

tensor([[0.4918, 0.5899, 0.0555,  ..., 1.4189, 1.5469, 1.7144],
        [0.1789, 0.5006, 0.4659,  ..., 1.0633, 1.3545, 0.5393],
        [0.2796, 0.1395, 0.0412,  ..., 0.5320, 0.4916, 0.8295],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.7130, 0.5586, 0.8045],
        [0.2357, 0.5359, 0.5710,  ..., 0.6700, 0.0000, 0.3146],
        [0.0000, 0.0662, 0.5751,  ..., 0.6450, 0.8230, 0.8644]],
       device='cuda:0', grad_fn=<IndexBackward>)

In [44]:
a = support.view(n_ways, k_shots, -1).mean(dim=1)

In [45]:
a.shape

torch.Size([5, 1600])

In [46]:
b = queries

In [50]:
n = a.shape[0]
m = b.shape[0]

In [64]:
input1 = torch.randn(100, 128)
input1.shape

torch.Size([100, 128])

In [ ]:
input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(input1, input2)

In [60]:
queries.unsqueeze(0).expand(n,m,-1)

tensor([[[0.0000, 0.0000, 0.1363,  ..., 0.7791, 0.6836, 0.4011],
         [0.6607, 0.6905, 0.4536,  ..., 1.0306, 1.2191, 1.4360],
         [0.0000, 0.0103, 0.1365,  ..., 0.0908, 0.3466, 0.5294],
         ...,
         [0.6833, 0.2566, 0.2808,  ..., 0.9656, 0.0514, 0.3068],
         [0.2939, 0.4591, 0.6109,  ..., 0.8635, 0.5289, 0.8151],
         [0.4777, 0.2959, 0.3257,  ..., 0.6681, 0.5542, 0.3793]],

        [[0.0000, 0.0000, 0.1363,  ..., 0.7791, 0.6836, 0.4011],
         [0.6607, 0.6905, 0.4536,  ..., 1.0306, 1.2191, 1.4360],
         [0.0000, 0.0103, 0.1365,  ..., 0.0908, 0.3466, 0.5294],
         ...,
         [0.6833, 0.2566, 0.2808,  ..., 0.9656, 0.0514, 0.3068],
         [0.2939, 0.4591, 0.6109,  ..., 0.8635, 0.5289, 0.8151],
         [0.4777, 0.2959, 0.3257,  ..., 0.6681, 0.5542, 0.3793]],

        [[0.0000, 0.0000, 0.1363,  ..., 0.7791, 0.6836, 0.4011],
         [0.6607, 0.6905, 0.4536,  ..., 1.0306, 1.2191, 1.4360],
         [0.0000, 0.0103, 0.1365,  ..., 0.0908, 0.3466, 0.

In [61]:
(a.unsqueeze(1).expand(n, m, -1) - queries.unsqueeze(0).expand(n,m,-1))**2

tensor([[[3.6123e-02, 1.1260e-01, 2.0007e-02,  ..., 7.4561e-02,
          4.5304e-02, 2.9212e-01],
         [2.2153e-01, 1.2597e-01, 3.0933e-02,  ..., 4.6551e-04,
          1.0405e-01, 2.4451e-01],
         [3.6123e-02, 1.0580e-01, 1.9951e-02,  ..., 9.2419e-01,
          3.0241e-01, 1.6983e-01],
         ...,
         [2.4324e-01, 6.2327e-03, 9.3467e-06,  ..., 7.4968e-03,
          7.1422e-01, 4.0293e-01],
         [1.0791e-02, 1.5251e-02, 1.1098e-01,  ..., 3.5604e-02,
          1.3514e-01, 1.5996e-02],
         [8.2713e-02, 1.5748e-03, 2.3003e-03,  ..., 1.4747e-01,
          1.1716e-01, 3.1607e-01]],

        [[8.5715e-02, 9.8171e-02, 5.0794e-02,  ..., 1.8140e-04,
          1.3637e-03, 2.7103e-02],
         [1.3539e-01, 1.4225e-01, 8.4547e-03,  ..., 5.6651e-02,
          2.4849e-01, 7.5748e-01],
         [8.5715e-02, 9.1829e-02, 5.0705e-02,  ..., 4.9246e-01,
          1.3987e-01, 1.3141e-03],
         ...,
         [1.5248e-01, 3.2159e-03, 6.5401e-03,  ..., 2.9931e-02,
          4.478

In [ ]:

logits = -((a.unsqueeze(1).expand(n, m, -1) -
            b.unsqueeze(0).expand(n, m, -1))**2).sum(dim=2)

In [7]:
train_tasks, valid_tasks, test_tasks, learner = setup('omniglot', '../../omniglot', 5, 5, 5, False, 0.03, 'cuda')
opt = optim.Adam(learner.parameters(), 0.01)
loss = nn.CrossEntropyLoss(reduction='mean')

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [8]:
ttask = train_tasks.sample()

In [9]:
ttask[0].shape

torch.Size([50, 1, 28, 28])

In [10]:
model = learner.clone()

In [11]:
data, labels = ttask
data, labels = data.to(device), labels.to(device)
total = n_ways * (k_shots + q_shots)
queries_index = np.zeros(total)

# Extracting the evaluation datums from the entire task set, for the meta gradient calculation 
for offset in range(n_ways):
    queries_index[np.random.choice(k_shots+q_shots, q_shots, replace=False) + ((k_shots + q_shots)*offset)] = True
support = data[np.where(queries_index == 0)]
support_labels = labels[np.where(queries_index == 0)]
queries = data[np.where(queries_index == 1)]
queries_labels = labels[np.where(queries_index == 1)]

# Inner adapt step
for _ in range(1):
    adapt_loss = loss(learner(support), support_labels)
    learner.adapt(adapt_loss)


In [12]:
preds = learner(queries)
preds

tensor([[-2.3058,  0.8712, -0.3009, -1.7466,  0.2174],
        [-1.1646,  0.6933,  0.0663, -0.8611,  0.2450],
        [-2.1604,  1.5395, -1.5346, -2.1959,  0.7367],
        [-1.6999,  0.4818, -1.1877, -2.2231, -0.6955],
        [-2.3995,  1.2572, -0.9198, -1.8828, -0.8643],
        [-3.2638,  0.4875, -0.9614, -0.7871,  0.4653],
        [-3.3043, -0.3927, -0.9587, -3.2453,  0.3632],
        [-1.9540, -2.1205, -1.3125, -1.1369, -0.3256],
        [-2.2095, -1.0025, -2.8084, -1.4914, -2.5339],
        [-3.6524, -1.3129,  0.4602, -2.3179,  1.8694],
        [-0.2180,  0.5564,  0.4151, -0.3363, -0.8863],
        [ 3.1093, -2.3283, -1.8008,  0.9621, -2.6799],
        [ 0.1015,  0.7979, -0.8207, -1.3333, -1.3204],
        [ 0.5794, -1.4207, -0.9276,  0.3640, -1.8067],
        [ 3.1537, -6.6684, -2.5053,  0.5397, -2.4689],
        [ 1.1471, -0.7925, -0.6154, -1.3091, -1.4252],
        [ 1.9084, -4.7103, -3.9228, -2.1826, -1.3713],
        [ 2.2197, -0.5845, -0.1133,  0.5664, -1.8442],
        [ 

In [13]:
eval_loss = loss(preds, queries_labels)

In [16]:
 preds.argmax(dim=1) == queries_labels

tensor([ True,  True,  True,  True,  True, False,  True,  True, False,  True,
        False, False, False, False, False,  True,  True,  True,  True,  True,
         True, False, False, False, False], device='cuda:0')

In [14]:
predictions = preds.argmax(dim=1).view(queries_labels.shape)
(predictions == queries_labels).sum().float() / queries_labels.size(0)

tensor(0.5600, device='cuda:0')

In [28]:
evaluation_loss, evaluation_accuracy = inner_adapt_maml(ttask, loss, model, 5,5,5, 1, device)

In [29]:
evaluation_loss.backward()

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling .backward() or autograd.grad() the first time.

In [27]:
for p in model.parameters():
    print(p.grad.data)

<ipython-input-27-8ed4d768114f>:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  print(p.grad.data)


AttributeError: 'NoneType' object has no attribute 'data'

In [32]:
evaluation_accuracy.item()

0.6800000071525574

In [36]:
np.arange(3).std()

0.816496580927726

In [38]:
name = 'anuj'

In [41]:
path = '../logs/{name}.csv' 
path

'../logs/{name}.csv'

In [6]:
profiler = Profiler('ProNets_{}_{}-shot_{}-way_{}-queries'.format('omni', 5,5,5))

In [87]:
name = 'omni'
prof = Profiler('abc_{}'.format(name))

In [7]:
profiler.log([2,2,3,4])

In [46]:
prof = Profiler('MAML_omni')

In [47]:
prof.log([1,2,3,4,5,6,7,8])

In [96]:
def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)

In [147]:
def set_Dataset(seed, mach, nways, kshots, tasks, data_loc):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    device = torch.device('cuda') if mach == 'cuda' else torch.device('cpu')

    data = l2l.vision.datasets.FullOmniglot(root=data_loc, transform=transforms.Compose([
                                                transforms.Resize(28),
                                                transforms.ToTensor()]))
    data = l2l.data.MetaDataset(data)
    transform = [
    l2l.data.transforms.NWays(data, n=2*nways),
    l2l.data.transforms.KShots(data, k=2*kshots),
    l2l.data.transforms.LoadData(data),
]
    tasksets = l2l.data.TaskDataset(data, transform, num_tasks=tasks)
    
    return tasksets, data


In [148]:
tasksets, data = set_Dataset(42, 'cuda', 5, 5, 20000, '../../omniglot')

In [157]:
len(data)

32460

In [158]:
# Load train/validation/test tasksets using the benchmark interface (5-way, 5-shot)
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                train_ways=5,
                                                train_samples=2*5,
                                                test_ways=5,
                                                test_samples=2*5,
                                                num_tasks=20000,
                                                root='../../omniglot',
)

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
Files already downloaded and verified
Files already downloaded and verified


In [161]:
len(tasksets.validation)

20000

In [46]:
tasksets.train.sample()[1].shape

torch.Size([50])

In [47]:
tasksets.test.sample()[0].shape

torch.Size([50, 1, 28, 28])

In [48]:
# Create model
model = l2l.vision.models.OmniglotFC(28 ** 2, 5)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=0.5, first_order=False) #wrapper on model for in-place weight updation (adaptation)
opt = optim.Adam(maml.parameters(), 0.003) #meta-optimizer
loss = nn.CrossEntropyLoss(reduction='mean')

In [49]:
maml

MAML(
  (module): OmniglotFC(
    (features): Sequential(
      (0): Flatten()
      (1): Sequential(
        (0): LinearBlock(
          (relu): ReLU()
          (normalize): BatchNorm1d(256, eps=0.001, momentum=0.999, affine=True, track_running_stats=False)
          (linear): Linear(in_features=784, out_features=256, bias=True)
        )
        (1): LinearBlock(
          (relu): ReLU()
          (normalize): BatchNorm1d(128, eps=0.001, momentum=0.999, affine=True, track_running_stats=False)
          (linear): Linear(in_features=256, out_features=128, bias=True)
        )
        (2): LinearBlock(
          (relu): ReLU()
          (normalize): BatchNorm1d(64, eps=0.001, momentum=0.999, affine=True, track_running_stats=False)
          (linear): Linear(in_features=128, out_features=64, bias=True)
        )
        (3): LinearBlock(
          (relu): ReLU()
          (normalize): BatchNorm1d(64, eps=0.001, momentum=0.999, affine=True, track_running_stats=False)
          (linear): 

In [68]:

def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)

    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True # set even indices to true
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)

    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy



In [90]:
learner = maml.clone()
batch = tasksets.train.sample()
evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                    learner,
                                                    loss,
                                                    1,
                                                    5,
                                                    5,
                                                    'cuda') # updates model inplace for inner update

evaluation_error.backward() # gradients comp on eval set for meta-update 

In [91]:
a=[]
for p in maml.parameters():
    a.append(p.grad.data)

torch.Size([256])
torch.Size([256])
torch.Size([256, 784])
torch.Size([256])
torch.Size([128])
torch.Size([128])
torch.Size([128, 256])
torch.Size([128])
torch.Size([64])
torch.Size([64])
torch.Size([64, 128])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64])
torch.Size([64])
torch.Size([5, 64])
torch.Size([5])


In [94]:
a

tensor([ 0.0839, -0.0497, -0.0046, -0.0065, -0.0231], device='cuda:0')

In [ ]:

for iteration in range(1):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0
    for task in range(32):
        # Compute meta-training loss
        learner = maml.clone()
        batch = tasksets.train.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device) # updates model inplace for inner update

        evaluation_error.backward() # gradients comp on eval set for meta-update 
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

        # Compute meta-validation loss
        learner = maml.clone()
        batch = tasksets.validation.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        meta_valid_error += evaluation_error.item()
        meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    print('Meta Valid Error', meta_valid_error / meta_batch_size)
    print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()

meta_test_error = 0.0
meta_test_accuracy = 0.0
for task in range(meta_batch_size):
    # Compute meta-testing loss
    learner = maml.clone()
    batch = tasksets.test.sample()
    evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                        learner,
                                                        loss,
                                                        adaptation_steps,
                                                        shots,
                                                        ways,
                                                        device)
    meta_test_error += evaluation_error.item()
    meta_test_accuracy += evaluation_accuracy.item()
print('Meta Test Error', meta_test_error / meta_batch_size)
print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)



In [ ]:

def main(
        ways=5,
        shots=1,
        meta_lr=0.003,
        fast_lr=0.5,
        meta_batch_size=32,
        adaptation_steps=1,
        num_iterations=60000,
        cuda=True,
        seed=42,
):
    
    



In [ ]:
if __name__ == '__main__':
    main()